# OLSZTYN
- scraping details 
- DATE 2024 jan 15th - this is done using URLs downloaded on the 4th of January

In [1]:
import os
import subprocess

# prepering directories
path = os.getcwd()
# subfolders
input_dir = os.path.join(path, 'inputs')
output_dir = os.path.join(path, 'outputs')

# open folder
subprocess.Popen(f'explorer "{output_dir}"')

In [2]:
# selenium 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# BeautifulSoup
from bs4 import BeautifulSoup

import datetime
import time

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()


Bad key "text.kerning_factor" on line 4 in
C:\Users\Ukasz\Anaconda3\envs\geopandas\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
data = 'olsztyn_page_all_pages_20240106151418.csv'
df = pd.read_csv(os.path.join(os.getcwd(), 'outputs', data))

In [4]:
# chrome driver location
PATH = os.path.join(path, 'inputs','chromedriver.exe' )

# chrom driver configuration
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless') # without opening browser

driver = webdriver.Chrome(PATH, options=options)

wait = WebDriverWait(driver, 10)

In [5]:
# url 
url = 'https://www.otodom.pl'
link = df['add_link'][0]
url + link

'https://www.otodom.pl/pl/oferta/apartament-premium-nad-jeziorem-olsztyn-mazury-ID4nqNc'

In [6]:
# open website using webdriver
driver.get(url + link)
print(f"Opens website: {url + link}")

Opens website: https://www.otodom.pl/pl/oferta/apartament-premium-nad-jeziorem-olsztyn-mazury-ID4nqNc


In [7]:
# accepting terms and conditions
# step One - pokaz cele
# this has to be done only once
element = wait.until(EC.element_to_be_clickable((By.ID, 'onetrust-pc-btn-handler')))
element.click()
# step two -  confirming preferences and closing the modal window
element = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[contains(text(), 'Potwierdzenie moich wyborów')]")))
element.click()

In [8]:
# url 
url = 'https://www.otodom.pl'
link = df['add_link'][8]
url + link

driver.get(url + link)
time.sleep(2) # waits n seconds
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [9]:
top_information = soup.find('div', attrs={'data-testid':'ad.top-information.table'})
date = datetime.datetime.now().strftime("%Y%m%d") # current date
with open(os.path.join(path, 'outputs', 'top_information' + date + '.html'), "w", encoding='utf-8') as file:
    file.write(str(top_information))

In [10]:
bottom_information = soup.find('div', attrs={'data-testid':'ad.additional-information.table'})

In [11]:
top_tags = ['Powierzchnia','Forma własności','Liczba pokoi','Stan wykończenia','Piętro',
       'Balkon / ogród / taras','Czynsz','Miejsce parkingowe','Obsługa zdalna'
        ,'Ogrzewanie']
bottom_tags = ['Rynek', 'Typ ogłoszeniodawcy', 'Dostępne od','Rok budowy',
                          'Rodzaj zabudowy','Okna','Winda','Media','Zabezpieczenia',
                         'Wyposażenie','Informacje dodatkowe','Materiał budynku']

In [12]:
data = {}
for tag in top_tags:
    try:
        div_tag = top_information.find('div', attrs={'aria-label':tag})
        results = div_tag.find_all("div")
        data[tag] = results[2].text
    except:
        data[tag] = results[2].text
    
data

{'Powierzchnia': '80,82 m²',
 'Forma własności': 'pełna własność',
 'Liczba pokoi': '4 ',
 'Stan wykończenia': 'do wykończenia',
 'Piętro': 'parter/1',
 'Balkon / ogród / taras': 'ogródek',
 'Czynsz': 'Zapytaj',
 'Miejsce parkingowe': 'Zapytaj',
 'Obsługa zdalna': 'Zapytaj',
 'Ogrzewanie': 'gazowe'}

In [13]:
for tag in bottom_tags:
    try:
        div_tag = bottom_information.find('div', attrs={'aria-label':tag})
        results = div_tag.find_all("div")
        data[tag] = results[2].text
    except:
        data[tag] = results[2].text

data

{'Powierzchnia': '80,82 m²',
 'Forma własności': 'pełna własność',
 'Liczba pokoi': '4 ',
 'Stan wykończenia': 'do wykończenia',
 'Piętro': 'parter/1',
 'Balkon / ogród / taras': 'ogródek',
 'Czynsz': 'Zapytaj',
 'Miejsce parkingowe': 'Zapytaj',
 'Obsługa zdalna': 'Zapytaj',
 'Ogrzewanie': 'gazowe',
 'Rynek': 'pierwotny',
 'Typ ogłoszeniodawcy': 'biuro nieruchomości ',
 'Dostępne od': 'brak informacji',
 'Rok budowy': '2023',
 'Rodzaj zabudowy': 'apartamentowiec',
 'Okna': 'plastikowe',
 'Winda': 'nie',
 'Media': 'telewizja kablowa, internet, telefon',
 'Zabezpieczenia': 'drzwi / okna antywłamaniowe, domofon / wideofon',
 'Wyposażenie': 'brak informacji',
 'Informacje dodatkowe': 'pom. użytkowe',
 'Materiał budynku': 'pustak'}

In [14]:
# adds data to the dataframe
for k in data:
    df.loc[8, k] = data[k]
df.to_csv(os.path.join(path, 'outputs', 'testttiiing.csv'), encoding='utf-8')